In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from warnings import filterwarnings
filterwarnings('ignore')
from tqdm import tqdm
plt.rcParams['figure.figsize'] = (10, 7)
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False

from lightgbm import LGBMRegressor
from sklearn.multioutput import MultiOutputRegressor

# 데이터 불러오기

In [3]:
train_x = pd.read_csv('./data/train_x_df.csv')
test_x = pd.read_csv('./data/test_x_df.csv')
train_y = pd.read_csv('./data/train_y_df.csv')

# 데이터 전처리

In [4]:
# 2차원 데이터를 3차원 데이터(sample_id, time, open)로 변환
x_train_open = pd.crosstab(index=train_x['sample_id'], columns=train_x['time'], values=train_x['open'], aggfunc='sum')
y_train_open = pd.crosstab(index=train_y['sample_id'], columns=train_y['time'], values=train_y['open'], aggfunc='sum')
x_test_open = pd.crosstab(index=test_x['sample_id'], columns=test_x['time'], values=test_x['open'], aggfunc='sum')

# 학습 및 예측
- 0 ~ 1379분의 시가를 학습 시켜 미래 0 ~ 119분의 시가를 예측합니다.
- 시가 외의 다른 특성을 추가하여 학습 해봤지만 오히려 성능이 떨어졌습니다.
- 여러 모델들을 사용한 결과 LGBMRegressor 모델이 가장 성능이 좋았고 파라미터 값을 바꾸면서 학습을 해봤지만 default 값에서 가장 성능이 좋았습니다.
- LGBMRegressor는 여러 개의 특성을 예측할 수 없기에 MultiOutputRegressor를 통해 학습시켰습니다.
- buy_quantity는 최초에는 예측 최대값이 1보다 클 때 전량 매수 했으나, 모든 경우에 대해 전량 매수할 때 결과가 더 좋게 나왔습니다.

In [10]:
model_lgbm = MultiOutputRegressor(LGBMRegressor())
model_lgbm.fit(x_train_open, y_train_open)
pred = model_lgbm.predict(x_test_open)

In [11]:
pred.shape

(535, 120)

# 제출용 데이터프레임 생성

In [13]:
submission = pd.DataFrame()
submission['sample_id'] = range(x_train_open.shape[0], x_train_open.shape[0]+x_test_open.shape[0])
submission['buy_quantity'] = 1
submission['sell_time'] = np.argmax(pred, axis=1)
submission

,sample_id,buy_quantity,sell_time
0,7661,1,44
1,7662,1,71
2,7663,1,101
3,7664,1,87
4,7665,1,107
...,...,...,...
530,8191,1,91
531,8192,1,117
532,8193,1,87
533,8194,1,101


In [14]:
# submission.to_csv('submission_lightgbm.csv', index=False)